In [1]:
# you get all pretrained model name here
# https://huggingface.co/transformers/pretrained_models.html
import pandas as pd
import os
import torch
import transformers
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification,EarlyStoppingCallback
# from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, EarlyStoppingCallback
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [2]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
# model_name = "bert-base-uncased"
# model_name = "distilbert-base-uncased"
model_name = 'vbert-base-pretrain'
model_dir = 'models-vbert-pretrain'
# max sequence length for each document/sentence sample
max_length = 512

In [3]:
tokenizer = BertTokenizerFast.from_pretrained(model_dir, do_lower_case=True)
# tokenizer = DistilBertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [4]:
# df = pd.read_csv('data/importance_detection_30_100000.csv')

In [5]:
# df.head()

In [6]:
def read_passages(path_data, test_size=0.2):
    df = pd.read_csv(path_data)
    documents = df['log'].to_list()
    labels_str = df['pcc'].to_list()
    labels_list = sorted(list(set(labels_str)))
    labels_all = {l:idx for idx, l in enumerate(labels_list)}
    labels = [labels_all[label_str] for label_str in labels_str]
    return train_test_split(documents, labels, test_size=test_size), labels_list
# end

In [7]:
# call the function
dir_data = 'data'
# name_data_file = 'importance_detection_30_2000_rand0.csv'
name_data_file = 'importance_detection_30_12000_rand0.csv'
path_data_relative = os.path.join(dir_data, name_data_file)
(train_texts, valid_texts, train_labels, valid_labels), target_names = read_passages(path_data_relative)

In [8]:
train_labels[0]

1

In [9]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)

In [10]:
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [11]:
train_dataset = NewsGroupsDataset(train_encodings, train_labels)
valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)

# dataloader_train = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
# dataloader_valid = torch.utils.data.DataLoader(valid_dataset, batch_size=8, shuffle=True)

In [12]:
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))
if len(os.listdir(model_dir)) > 0:
    print('load model from local')
    model_info = model_dir
else:
    print('load model from official')
    model_info = model_name
    
# model = DistilBertForSequenceClassification.from_pretrained(model_info)
model = BertForSequenceClassification.from_pretrained(model_info)

if torch.cuda.is_available():
    model = model.cuda()
# end

load model from local


Some weights of the model checkpoint at models-vbert-pretrain were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not init

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids.reshape(-1)
    # pred = np.argmax(pred, axis=1)
    preds = pred.predictions.argmax(-1).reshape(-1)

#     print('labels: {}'.format(labels))
#     print('pred: {}'.format(preds))
    accuracy = accuracy_score(y_true=labels, y_pred=preds)
    precision = precision_score(y_true=labels, y_pred=preds, zero_division=1)
    recall = recall_score(y_true=labels, y_pred=preds, zero_division=1)
    f1 = f1_score(y_true=labels, y_pred=preds, zero_division=1)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [14]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=1,               # log & save weights each logging_steps
    evaluation_strategy="epoch",     # evaluate each `logging_steps`
    learning_rate=2e-5,
    save_strategy='epoch',
    save_total_limit=3,
    metric_for_best_model='f1'
)

In [15]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [16]:
trainer.train()

***** Running training *****
  Num examples = 9581
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 11980


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000500,0.015378,0.995826,0.995823,1.000000,0.997907
2,0.000100,0.026145,0.997078,0.997072,1.000000,0.998534
3,0.000500,0.034783,0.995409,0.995407,1.000000,0.997698
4,0.000400,0.012434,0.998331,0.998325,1.000000,0.999162
5,0.000100,0.019138,0.996661,0.996656,1.000000,0.998325
6,0.000100,0.011477,0.998748,0.998743,1.000000,0.999371
7,0.000100,0.017068,0.997913,0.997907,1.000000,0.998952
8,0.000200,0.013617,0.998331,0.998325,1.000000,0.999162
9,0.000000,0.009992,0.998748,0.999161,0.999581,0.999371


***** Running Evaluation *****
  Num examples = 2396
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1198
Configuration saved in ./results/checkpoint-1198/config.json
Model weights saved in ./results/checkpoint-1198/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2396
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-2396
Configuration saved in ./results/checkpoint-2396/config.json
Model weights saved in ./results/checkpoint-2396/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2396
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-3594
Configuration saved in ./results/checkpoint-3594/config.json
Model weights saved in ./results/checkpoint-3594/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2396
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-4792
Configuration saved in ./results/checkpoint-4792/config.json
Model weights saved in ./results/checkpoint-4792/pytorch_model.

TrainOutput(global_step=10782, training_loss=0.008308707701905326, metrics={'train_runtime': 3288.7127, 'train_samples_per_second': 29.133, 'train_steps_per_second': 3.643, 'total_flos': 2.268780319263744e+16, 'train_loss': 0.008308707701905326, 'epoch': 9.0})

In [17]:
# trainer.evaluate()

In [18]:
model.save_pretrained('./models-vbert-finetune')

Configuration saved in ./models-vbert-finetune/config.json
Model weights saved in ./models-vbert-finetune/pytorch_model.bin
